This file, once complete, will obtain a dataframe that contains every trade that occured during the 2022-23 NHL season. I am going to scrape the CapFriendly Website using beautiful soup and selenium and then use pandas to get it into the dataframe.

In [1]:
import requests
import pandas as pd
import time
import logging
import lxml
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

Now that necessary packages are installed, going to parse the needed url. This URL has a dynamic table that contains the cost per save of NHL Goalies for the 2019-20 season. So I am going to scrape the cap friendly website (Cost Per Save tab) and get this into a dataframe that I can eventually download and put into the MySQL database. As mentioned, since the table is dynamic and not static, the selenium package is needed because the table only expands to its full length once you scroll down.

In [2]:
# Set up the Selenium driver (make sure you have the correct path to your WebDriver)
driver_path = 'C:/STL Blues Analytics/SalaryCap/NHLSalaryCap/chromedriver.exe'  # Update this to your WebDriver path
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Open the webpage
url = "https://www.capfriendly.com/cost-per-save/2020/season"
driver.get(url)

# Get initial page height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for new content to load
    time.sleep(2)
    
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break
    last_height = new_height

# Get the page source after all rows are loaded
page_source = driver.page_source
driver.quit()

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find the table and convert it to a DataFrame
table = soup.find('table')
dfattempt = pd.read_html(str(table))[0]



C:\Users\nsofi\AppData\Local\Temp\ipykernel_8488\2678146920.py:36: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfattempt = pd.read_html(str(table))[0]


In [3]:
dfattempt.shape
dfattempt.tail(5)

,#,GOALIE,AGE,TYPE,EXPIRY,CAP HIT,P. BONUSES,GP,MIN,SH,SV,SV%,GAA,$/GP,$/MIN,$/SV
67,68,"Bobrovsky, Sergei",31,NaN,Stnd,UFA,"$10,000,000",50,2805,1505,1354.0,0.900,3.23,"$200,000","$3,565","$7,385"
68,69,"Kivlenieks, Matiss",23,NaN,ELC,RFA,"$925,000",6,284,137,123.0,0.898,2.96,"$154,166","$3,257","$7,520"
69,70,"Kinkaid, Keith",30,NaN,Stnd,UFA,"$1,750,000",6,339,192,168.0,0.875,4.25,"$291,666","$5,162","$10,416"
70,71,"Lundqvist, Henrik",37,NaN,Stnd,UFA,"$8,500,000",30,1596,883,799.0,0.905,3.16,"$283,333","$5,325","$10,638"
71,72,"Schneider, Cory",33,NaN,Stnd,UFA,"$6,000,000",13,679,355,315.0,0.887,3.53,"$461,538","$8,836","$19,047"


In [4]:
dfattempt.to_csv("CF Cost Per Save 2019-20.csv") 